# Giovedì 30 Giugno 2022
Abbiamo creato un nuovo modulo, chiamato bullet.py:


In [ ]:
#bullet.py

import pygame

class Bullet(pygame.sprite.Sprite):

    def __init__(self, type, pos, direction):
        super().__init__()
        self.type = type



        self.damage = 1
        self.speed = 10
        self.direction = pygame.math.Vector2(0, 0)
        self.direction = direction

        # zona setup che dipende dal TYPE
        if self.type == "9mm":
            self.image = pygame.image.load("img/bullet.png")
            self.original_image = self.image
            self.speed = 30
            self.damage = 1
        elif self.type == "heavy":
            self.image = pygame.image.load("img/bullet.png")
            self.original_image = self.image
            self.speed = 10
            self.damage = 3
        else:
            self.image = pygame.image.load("img/bullet.png")
            self.original_image = self.image
            self.speed = 30
            self.damage = 1

        self.rect = self.image.get_rect()
        self.rect.topleft = pos

    def update(self):
        self.rect.x += self.speed * self.direction[0]
        self.rect.y += self.speed * self.direction[1]

In [ ]:
# enemy.py

import pygame.sprite
import math


class Enemy(pygame.sprite.Sprite):

    def __init__(self, image, pos):
        super().__init__()
        self.image = image
        self.original_image = image
        self.rect = self.image.get_rect()
        self.rect.topleft = pos

        self.speed = 1
        self.direction = pygame.math.Vector2(0, 0)
        self.health = 2
        self.type = type

    def update(self):
        self.rect.y -= self.speed

è necessario poi controllare le collisioni tra i proiettili e gli zombie. Nel main:

In [ ]:
#collision control area
for zombie in enemy_group:
    if zombie.rect.colliderecr(hero.rect):
        hero.damage()
        if hero.get_health() <= 0:
            pygame.quit()
            sys.exit()

for bullet in player_bullets_group: #per ogni proiettile, vedi se colpisce qualsiasi nemico
    for zombie in enemy_group:
        if bullet.rect.colliderect(zombie):
            zombie.deal_damage() #dobbiamo creare questa funzione!


Vogliamo che quando il proiettile colpisce lo zombie:
- ci sia un danno allo zombie
- il colpo emetta un suono
- il proiettile muoia,

Introduciamo la funzione deal damage su enemy.py, ma anche gli effetti sonori.

In [ ]:
    # audio files (def init)
    self.hit_sound = pygame.mixer.Sound("sound/bullet_hit.mp3")
    self.death_sound = pygame.mixer.Sound("sound/zombie_hit.mp3")

def deal_damage(self, amount = 1):
    self.health -= amount
    self.hit_sound.play()
    if self.health <= 0:
        self.die()

def die(self):
    self.death_sound.play()
    self.kill()

in main.py:

In [ ]:
for live_bullet in player_bullets_group:
    for zombie in enemy_group:
        if live_bullet_rect.colliderect(zombie):
            zombie.deal_damage()
            live_bullet.kill()

Facciamo uno script che ci generi diversi tipi di zombie. Modifichiamo la classe enemy:

In [ ]:
class Enemy(pygame.sprite.Sprite):

    def __init__(self, image, pos, type = "normal"):
        super().__init__()
        self.image = pygame.transform.scale(image,(64,71))
        self.original_image = self.image
        self.rect = self.image.get_rect()
        self.rect.topleft = pos

        #game attributes
        self.speed = 1
        self.direction = pygame.math.Vector2(0, 0)
        self.health = 2
        self.type = type
        
        # audio files (def init)
        self.hit_sound = pygame.mixer.Sound("sound/bullet_hit.mp3")
        self.death_sound = pygame.mixer.Sound("sound/zombie_hit.mp3")

        #type variations
        if self.tupe == "fast":
            self.speed *= 2
            self.health /= 2

        def update(self):
            self.rect.y -= self.speed

def deal_damage(self, amount = 1):
    self.health -= amount
    self.hit_sound.play()
    if self.health <= 0:
        self.die()

def die(self):
    self.death_sound.play()
    self.kill()

Rendiamo il gioco più difficile.
Serve un modulo che permetta allo zombie di seguire il giocatore.
Modifichiamo quindi la funzione update in enemy.py:

In [ ]:
import player

def update(self,current_player):
    victim = player.Player(self.image,self.rect.center) #sto creando un giocatore che ha l'immagine dello zombie e il rettangolo dello zombie
    victim = current_player 
    self.angle = self.get_player_angle(victim.rect.centerx, victim.rect.centery) #perchè lo zombie deve guardare la x e la y del giocatore
    self.rot_center(self.angle)

    self.rect.x += self.speed * self.direction.x
    self.rect.y += self.speed * self.direction.y


def rot_center(self,angle):
    pass #c'è qualcosa

def get_player_angle(self,x,y):
    distance_x = x -self.rect.centerx
    distance_y = y- self.rect.centery
    angle = math.atan2(distance_y, distance_x)
    look_angle = -angle*180/math.pi
    sprite_rotation = -90
    return look_angle + sprite_rotation

Così lo zombie guarda il nostro giocatore, ma non lo insegue. Quindi dobbiamo apportare una modifica. Se io modifico in enemy.py in game attributions:

In [ ]:
self.direction = pygame.math.vector2(0,0)

In

In [ ]:
self.direction = pygame.math.vector2(0,-1)

Allora lo zombie andrà costantemente verso l'alto, girandosi e inseguendo con lo sguardo il nostro giocatore. Ma noi vogliamo che lo zombie segua il nostro giocatore!

Modifichiamo la funzione get player angle in enemy.py:

In [ ]:
def get_player_angle(self,x,y):
    distance_x = x -self.rect.centerx
    distance_y = y- self.rect.centery


    #section for calculating the direction vector
    if distance_x > 0:
        self.direction.x = 1
    elif distance_x <0:
        self.direction.x = -1
    else:
        self.direction.x = 0

    if distance_y > 0:
        self.direction.y = 1
    elif distance_y <0:
        self.direction.y = -1
    else:
        self.direction.y = 0

    angle = math.atan2(distance_y, distance_x)
    look_angle = -angle*180/math.pi
    sprite_rotation = -90
    return look_angle + sprite_rotation

Ora lo zombie segue il giocatore.

Però ora vogliamo creare un nuovo file, che chiameremo settings.py. Non ci sarà del vero e proprio codice, ma terremo proprio delle impostazioni del gioco, come la grandezza dello schermo.
Ora i file sono:

- main.py
- player.py
- enemy.py
- ui.py
- bullet.py
- settings.py


In [ ]:
# settings.py

screen_width = 800
screen_height = 600

Nel main importiamo settings e cambiamo la dicitura della variabile screen:

In [ ]:
# main.py
##
## 
import settings
##
##

screen = pygame.display.set_mode((settings.screen_width, settings.screen_height))

Ora vogliamo creare uno script per generare un sacco di nemici. Creiamo un nuovo modulo, chiamato spawner.py.

Ora i file sono:

- main.py
- player.py
- enemy.py
- ui.py
- bullet.py
- settings.py
- spawner.py


In [ ]:
# spawner.py

import random
import enemy
import settings

MIN_SPAWN_RANGE = 64
MAX_SPAWN_RANGE = 128

def random_spawn():
    type = random.choice(enemy.Enemy.ZOMBIE_TYPES)
    x = settings.screen_widt
    y = settings.screen_height

    coint_toss = random.randint(0,1)

    # 0 = heads -> spawn right

    if coin_toss == 0:
        min = settings.screen_width + MIN_SPAWN_RANGE
        max = settings.screen_width + MAX_SPAWN_RANGE
        x = random.randint(min,max)

    #1 = tails -> spawn left
    elif coin_toss ==1:
        min = 0 - MIN_SPAWN_RANGE
        max = 0 - MAX_SPAWN_RANGE
        x = random.randint(min,max)

    coin_toss = random.randint(0,1)
        # 0 = heads -> spawn down

    if coin_toss == 0:
        min = settings.screen_height + MIN_SPAWN_RANGE
        max = settings.screen_height + MAX_SPAWN_RANGE
        x = random.randint(min,max)

    #1 = tails -> spawn up
    elif coin_toss ==1:
        min = 0 - MIN_SPAWN_RANGE
        max = 0 - MAX_SPAWN_RANGE
        y = random.randint(min,max)

    
    pos = (x,y)
    return enemy.Enemy(pos=pos, type = type)

Ora in main.py introduciamo una variabile chiamata enemy_timer = 0. Dopodich+ possiamo introdurre una #spawn area.

In [ ]:
#spawn area
enemy_timer -= 1
if enemy_timer <0:
    enemy_group.add(spawner.random_spawn())

### Programmi finiti il 30 giugno

In [ ]:
# main

import math
import random
import sys
import pygame
import settings
import bullet
import enemy
import player

# per far partire i processi di pygame, devo far partire l' init
import spawner
import ui

pygame.init()
clock = pygame.time.Clock()
pygame.display.set_caption("Montecreto Mount of Madness")
screen = pygame.display.set_mode((settings.screen_width, settings.screen_height))

# music setup
pygame.mixer.music.load("sound/survival.mp3")
pygame.mixer.music.set_volume(0.1)
pygame.mixer.music.play(-1)
gunshot = pygame.mixer.Sound("sound/Revolver Shoot.mp3")

# image zone
image = pygame.image.load("img/player.png")
image = pygame.transform.scale(image, (60, 32))
hero = player.Player(image, (400, 300))
interface = ui.Ui(screen, hero)

zombie = enemy.Enemy((400, 500))

# sprite zone
player_group = pygame.sprite.Group()
player_group.add(hero)
enemy_group = pygame.sprite.Group()
enemy_group.add(zombie)
player_bullets_group = pygame.sprite.Group()

enemy_timer = 0

# showcase


# gameplay cycle
while True:
    # INPUT CONTROL AREA
    mouse_x, mouse_y = pygame.mouse.get_pos()
    for event in pygame.event.get():
        # key press event
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT or event.key == pygame.K_d:
                hero.direction.x = 1
            elif event.key == pygame.K_LEFT or event.key == pygame.K_a:
                hero.direction.x = -1
            elif event.key == pygame.K_UP or event.key == pygame.K_w:
                hero.direction.y = -1
            elif event.key == pygame.K_DOWN or event.key == pygame.K_s:
                hero.direction.y = +1
        # key lift event
        if event.type == pygame.KEYUP:
            if (event.key == pygame.K_RIGHT or event.key == pygame.K_d) and hero.direction.x == 1:
                hero.direction.x = 0
            elif (event.key == pygame.K_LEFT or event.key == pygame.K_a) and hero.direction.x == -1:
                hero.direction.x = 0
            elif (event.key == pygame.K_UP or event.key == pygame.K_w) and hero.direction.y == -1:
                hero.direction.y = 0
            elif (event.key == pygame.K_DOWN or event.key == pygame.K_s) and hero.direction.y == 1:
                hero.direction.y = 0
        # mouse press event
        if event.type == pygame.MOUSEBUTTONDOWN:
            # pressing left mouse button
            if event.button == 1:
                gunshot.play()
                type = hero.held_weapon()
                angle = hero.get_shoot_angle()
                x = math.cos(angle)
                y = math.sin(angle)
                direction = (x, y)
                #originX = hero.rect.centerx
                #originY = hero.rect.centery
                origin = hero.get_shoot_pos()
                new_bullet = bullet.Bullet(type=type, pos=origin, direction=direction)
                player_bullets_group.add(new_bullet)

        # se l'evento è di tipo QUIT...
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # collision control area
    for zombie in enemy_group:
        if zombie.rect.colliderect(hero.rect):
            hero.damage()
            if hero.get_health() <= 0:
                pygame.quit()
                sys.exit()

    for live_bullet in player_bullets_group:
        for zombie in enemy_group:
            if live_bullet.rect.colliderect(zombie):
                zombie.deal_damage()
                live_bullet.collide()


    # spawn area
    enemy_timer -= 1
    if enemy_timer <0:
        enemy_group.add(spawner.random_spawn())
        enemy_timer = random.randint(60, 120)


    # update area
    player_group.update(mouse_x, mouse_y)
    enemy_group.update(hero)
    player_bullets_group.update()

    # draw area --- background goes first, then the rest --- at the bottom goes the UI
    screen.fill("#45d1a0")
    enemy_group.draw(screen)
    player_group.draw(screen)
    player_bullets_group.draw(screen)
    interface.draw()
    pygame.display.update()
    clock.tick(60)

In [ ]:
#player.py

import pygame.sprite
import math
from pygame import mixer

class Player(pygame.sprite.Sprite):

    def __init__(self, image, pos):
        super().__init__()
        self.image = image
        self.original_image = self.image
        self.rect = self.image.get_rect()
        self.rect.topleft = pos

        # game attributes
        self.speed = 5
        self.direction = pygame.math.Vector2(0, 0)
        self.health = 3
        self.grace_frames = 60
        self.grace_timer = 60
        self.weapon = "9mm"
        self.angle = 90
        self.shoot_angle = 0

        # audio files
        self.oof = mixer.Sound("sound/oof.mp3")

    def update(self, mouse_x, mouse_y):
        self.grace_timer += 1
        self.angle = self.get_mouse_angle(mouse_x, mouse_y)
        self.rot_center(self.angle)
        self.rect.x += self.speed * self.direction.x
        self.rect.y += self.speed * self.direction.y

    def rot_center(self, angle):
        x = self.rect.centerx
        y = self.rect.centery
        image = self.image
        rotated_image = pygame.transform.rotate(self.original_image, angle)
        new_rect = rotated_image.get_rect(center=image.get_rect(center=(x, y)).center)

        self.image = rotated_image
        self.rect = new_rect

    def get_mouse_angle(self, x, y):
        distance_x = x - self.rect.centerx
        distance_y = y - self.rect.centery
        angle = math.atan2(distance_y, distance_x)
        look_angle = -angle*180/math.pi
        self.shoot_angle = angle
        return look_angle

    def damage(self, amount = 1):
        if self.grace_timer >= self.grace_frames:
            self.health -= amount
            self.grace_timer = 0
            self.oof.play()

    def get_health(self):
        return self.health

    def held_weapon(self):
        return self.weapon

    def get_direction(self):
        return self.direction

    def get_angle(self):
        return self.angle

    def get_shoot_angle(self):
        return self.shoot_angle

    def get_shoot_pos(self):
        pos = pygame.math.Vector2(0,0)
        angle = self.shoot_angle
        offsetx = self.image.get_size()[0]/6 * math.cos(angle)
        offsety = self.image.get_size()[1]/6 * math.sin(angle)
        pos.x = self.rect.centerx + offsetx
        pos.y = self.rect.centery + offsety
        return pos


In [ ]:
# enemy.py

import pygame.sprite
import math
import player


class Enemy(pygame.sprite.Sprite):

    ZOMBIE_TYPES = ["normal", "fast"]

    def __init__(self, pos, type="normal"):
        super().__init__()


        # game attributes
        self.speed = 1
        self.direction = pygame.math.Vector2(0, 0)
        self.health = 2
        self.type = type

        # audio files
        self.hit_sound = pygame.mixer.Sound("sound/bullet_hit.mp3")
        self.death_sound = pygame.mixer.Sound("sound/zombie_hit.mp3")

        # type variations
        if self.type == "fast":
            self.speed *= 3
            self.health /= 2
            self.image = pygame.image.load("img/zombie.png")
            self.image = pygame.transform.scale(self.image, (43, 51))
        else:
            self.image = pygame.image.load("img/zombie.png")
            self.image = pygame.transform.scale(self.image, (64, 71))


        self.original_image = self.image
        self.rect = self.image.get_rect()
        self.rect.topleft = pos
        self.angle = 0

    def update(self, current_player):
        victim = current_player
        self.angle = self.get_player_angle(victim.rect.centerx, victim.rect.centery)
        self.rot_center(self.angle)
        self.rect.x += self.speed * self.direction.x
        self.rect.y += self.speed * self.direction.y

    def deal_damage(self, amount=1):
        self.health -= amount
        self.hit_sound.play()
        if self.health <= 0:
            self.die()

    def die(self):
        self.death_sound.play()
        self.kill()

    def rot_center(self, angle):
        x = self.rect.centerx
        y = self.rect.centery
        image = self.image
        rotated_image = pygame.transform.rotate(self.original_image, angle)
        new_rect = rotated_image.get_rect(center=image.get_rect(center=(x, y)).center)

        self.image = rotated_image
        self.rect = new_rect

    def get_player_angle(self, x, y):
        distance_x = x - self.rect.centerx
        distance_y = y - self.rect.centery

        # section for calculating the direction vector
        if distance_x > 0:
            self.direction.x = 1
        elif distance_x < 0:
            self.direction.x = -1
        else:
            self.direction.x = 0

        if distance_y > 0:
            self.direction.y = 1
        elif distance_y < 0:
            self.direction.y = -1
        else:
            self.direction.y = 0

        angle = math.atan2(distance_y, distance_x)
        look_angle = -angle*180/math.pi
        sprite_rotation = -90
        return look_angle + sprite_rotation





In [ ]:
# ui.py

import pygame
import player

class Ui():

    def __init__(self, screen, hero):
        self.score = 0
        self.font = pygame.font.Font("freesansbold.ttf", 60)
        self.screen = screen
        self.hero = hero

    def draw(self):
        font = self.font
        screen = self.screen

        health_counter = font.render(f"HP: {self.hero.get_health()}", True, (255, 255, 255))
        screen.blit(health_counter, (0, 0))


In [ ]:
# bullet.py

import pygame

class Bullet(pygame.sprite.Sprite):

    def __init__(self, type, pos, direction):
        super().__init__()
        self.type = type



        self.damage = 1
        self.speed = 10
        self.direction = pygame.math.Vector2(0, 0)
        self.direction = direction

        # zona setup che dipende dal TYPE
        if self.type == "9mm":
            self.image = pygame.image.load("img/bullet.png")
            self.original_image = self.image
            self.speed = 30
            self.damage = 1
        elif self.type == "heavy":
            self.image = pygame.image.load("img/bullet.png")
            self.original_image = self.image
            self.speed = 10
            self.damage = 3
        else:
            self.image = pygame.image.load("img/bullet.png")
            self.original_image = self.image
            self.speed = 30
            self.damage = 1

        self.rect = self.image.get_rect()
        self.rect.topleft = pos

    def update(self):
        self.rect.x += self.speed * self.direction[0]
        self.rect.y += self.speed * self.direction[1]

    def collide(self):
        self.kill()

In [ ]:
# settings.py
screen_width = 800
screen_height = 600

In [ ]:
# spawner.py

import pygame
import random
import enemy
import settings

MIN_SPAWN_RANGE = 64
MAX_SPAWN_RANGE = 128

def random_spawn():
    type = random.choice(enemy.Enemy.ZOMBIE_TYPES)
    x = settings.screen_width
    y = settings.screen_height

    coin_toss = random.randint(0, 1)
    random_range = random.randint(MIN_SPAWN_RANGE, MAX_SPAWN_RANGE)
    # 0 = heads -> spawn right
    if coin_toss == 0:

        x = settings.screen_width + random_range
    # 1 = tails -> spawn left
    elif coin_toss == 1:

        x = 0 - random_range

    coin_toss = random.randint(0, 1)
    random_range = random.randint(MIN_SPAWN_RANGE, MAX_SPAWN_RANGE)
    # 0 = heads -> spawn down
    if coin_toss == 0:

        y = settings.screen_height + random_range
    # 1 = tails -> spawn up
    elif coin_toss == 1:

        y = 0 - random_range


    pos = (x, y)
    return enemy.Enemy(pos=pos, type=type)